In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.read_csv("housedata.csv")
df.drop(['SizeRank', 'RegionType', 'RegionName', 'StateName', 'Metro', 'CountyName'], axis=1, inplace=True)
cols = list(df.columns)
#df.dropna(inplace=True)
rows = df.shape[0]

In [ ]:
for row in range(rows):
  for idx, col in enumerate(cols):
    if col in ['RegionID', 'State', 'City']:
      continue
    if pd.isna(df.at[row, col]):
      while(pd.isna(df.at[row, col])):
        try:
          df.loc[row, col] = np.nanmin(df.iloc[row, idx+1:].values)
        except:
          df.loc[row, col] = np.nanmax(df.iloc[row, :idx].values)
        print(f'Replaced NA at row {row} and col {idx} with value {df.at[row, col]}')